# 🤖 Classic ML Models with Benchmark Evaluation 📈

<a id="section-zero"></a>
# Introduction
Banks are often exposed to fraudulent transactions and they're constantly improving systems to track them. My goal is to train eight (8) machine learning models that will detect those fraudulent transactions.

The models will be trained using a bank dataset that contains 20k+ transactions with 112 anonymized numerical features.

In [ ]:
import os
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

Read and display data.

In [ ]:
# Set working directory and read data
data = pd.read_csv('dataset/fraud_detection_bank_dataset.csv') 
data

Drop the first column and rename the last column. Check for any null values.

In [ ]:
# Remove the 1st column (Unnamed).
data = data.iloc[:,1:]

# Rename "targets" column to "is_fraud".
data.rename(columns={'targets': 'is_fraud'}, inplace=True) 

# Check for any null values in each column
print(f'Null Values: {sum(data.isnull().sum())}')

Determine the class balance between non-fraudulent and fraudulent objects.

In [ ]:
# Obtain the number of fraud & non-fraud objects.
class_balance = data.is_fraud.value_counts()
fraud_percentage = ((class_balance[1]/len(data))*100)

print(f'Non-Fraud: {class_balance[0]} '
      f'({(100-fraud_percentage).__round__(4)}%)')
print(f'Fraud: {class_balance[1]} '
      f'({fraud_percentage.__round__(4)}%)')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Compute the correlation matrix
corrmat = data.corr()

# Select the top 30 features most correlated with 'is_fraud'
k = 30
cols = corrmat.nlargest(k, 'is_fraud')['is_fraud'].index

# Compute the correlation matrix for these selected features
cm = np.corrcoef(data[cols].values.T)

# Set the style and generate the heatmap
plt.figure(figsize=(14, 12))
sns.set(font_scale=1.1)
sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f',
            annot_kws={'size': 9}, yticklabels=cols.values, xticklabels=cols.values,
            cmap='coolwarm')

plt.title('Correlation Among the Top 30 Features Most Related to "is_fraud"')
plt.savefig('img/correlation_heatmap_top_30_features.png', bbox_inches='tight')
plt.show()


Separate dataset into features (inputs) and targets (outputs). Verify all features are numerical and split dataset into training (80%) and testing (20%).

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# SMOTE the dataset
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X = data.drop('is_fraud', axis=1)
y = data['is_fraud']
X, y = smote.fit_resample(X, y)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Initialize functions that later will be used to analyze data.

In [ ]:
def get_indices(lst, val):
    '''
    Input: 
        lst: a list of elements (string, integer, float, etc).
        val: the value being searched for in the list.
        
    Output: 
        indices: list of all indices that are equal to val.
    '''
    # Gets all the indices of a element in a list.
    indices = []
    
    for i in range(len(lst)):
        if lst[i] == val:
            indices.append(i)
            
    return indices

In [ ]:
def get_bestParam(param_values, train_acc, test_acc):
    '''
    Input: 
        param_values: a list containing all of the parameter values tried
        train_acc: a list containing the training accuracy for a model trained 
                   with each parameter in param_values.
        test_acc: a list containing testing accuracy for a model trained with 
                  each parameter in param_values.
        
    Output: 
        best_param: the alpha value that produces the highest test_acc with 
                    the minimum difference (train_acc - test_acc).
        best_train_acc: the training value that is the closest to the best_test_acc.
        best_test_acc: the highest testing accuracy.
    '''
    # Gets all the indices of the maximum test accuracy value.
    indices = get_indices(test_acc, max(test_acc))

    diff = []
    params = []
    train_accuracy = []
    test_accuracy = []
    
    for i in indices:
        # Calculate the difference between the training and testing 
        # accuracy based on the indices.
        diff.append(abs(train_acc[i] - test_acc[i]))
        params.append(param_values[i])
        train_accuracy.append(train_acc[i])
        test_accuracy.append(test_acc[i])
        
    # Get the index of the minimum difference in the list.
    min_diff_idx = diff.index(min(diff))
    # Get the best param using the minimum difference index.
    best_param = params[min_diff_idx]
    # Get the best training accuracy using the minimum difference index.
    best_train_acc = train_accuracy[min_diff_idx] 
    # Get the best testing accuracy using the minimum difference index.
    best_test_acc = test_accuracy[min_diff_idx] 
    
    return best_param, best_train_acc, best_test_acc

In [ ]:
def disp_confMatrix():
    cm = confusion_matrix(y_test, y_test_predicted)
    cmp = ConfusionMatrixDisplay(cm, display_labels=target_names)
    fig, ax = plt.subplots(figsize=(7,7))
    cmp.plot(ax=ax)
    plt.show()
    
    return None

In [ ]:
def save_metrics():
    # Calculate accuracy for training and testing data.
    train_accs.append(accuracy_score(y_train, y_train_predicted).__round__(4))
    test_accs.append(accuracy_score(y_test, y_test_predicted).__round__(4))

    # Calculate F1 score for training and testing data.
    train_f1_scores.append(f1_score(y_train, y_train_predicted).__round__(4))
    test_f1_scores.append(f1_score(y_test, y_test_predicted).__round__(4))

    # Obtain the false-positives and false-negatives from the testing data.
    false_pos.append(confusion_matrix(y_test, y_test_predicted)[0,1]) 
    false_neg.append(confusion_matrix(y_test, y_test_predicted)[1,0]) 
    
    return None

Initialize lists for storing metrics.

In [ ]:
target_names = ['non-fraud (0)', 'fraud (1)'] # Used for classification report.
false_pos = [] # Create list for false-positives.
false_neg = [] # Create list for false-negatives.
train_f1_scores = [] # Create list for training data F1 scores.
test_f1_scores = []  # Create list for testing data F1 scores.
train_accs = [] # Create list for training data accuracies.
test_accs = []  # Create list for testing data accuracies.
clf_names = ['Dummy', 'Decision Tree', 'AdaBoost', 'KNN', 'MLP', 
             'Logistic\nRegression', 'Random\nForest', 'SVM']

<a id="section-one"></a>
# Training & Testing ML Models

I will be training and testing eight (8) different ML models using *k*-fold cross-validation (CV) with hyper-parameter tuning to reduce the chances of overfitting.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import auc
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve
from sklearn.model_selection import validation_curve

# Cross-validation value.
cv = 3 

<a id="section-two"></a>
## Dummy Classifier
This model was configured to always pick the majority class. I used this as a baseline to determine if the other models were doing better than a naive approach that wouldn't require any real training. 

In [ ]:
from sklearn.dummy import DummyClassifier

Train and fit data to the model. Predict training and testing class. Display the classification report.

In [ ]:
# Create model and fit model to the training data.
dum = DummyClassifier(strategy='most_frequent', random_state=random_seed)
dum.fit(X_train, y_train)

# Predict training & testing class.
y_train_predicted = dum.predict(X_train)
y_test_predicted = dum.predict(X_test)

# Display classification report for training & testing data.
print(f"Classification Report with strategy = 'most_frequent'\n")
print(f'\t\t    Dummy Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print(f'\t\t    Dummy Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4))

Calculate probability score, precision and recall. Then, plot the AUC-PR.

*Note: AUC-PR stands for area under the (precision-recall) curve. Generally, the higher the AUC-PR score, the better a classifier performs for the given task.*

In [ ]:
# Predict class probabilities for testing data.
y_score = dum.predict_proba(X_test)[:, 1] 

# Compute precision-recall pairs for different probability thresholds.
dum_precision, dum_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
dum_auc = auc(dum_recall, dum_precision).__round__(4)
              
plt.figure(figsize=(15,8))            
plt.plot(dum_recall, dum_precision, color='black', linestyle='--')
plt.legend([f'Dummy (AUC = {dum_auc})'])
plt.title(f'Dummy Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

Store metrics to the list.

In [ ]:
save_metrics()

<a id="section-three"></a>
## Decision Tree Classifier
### Choosing a `ccp_alpha` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Create model.
dtc = DecisionTreeClassifier(criterion="gini", random_state=random_seed)

# Generates a list that starts at minimum, ends at maximum, and increments by step.
alpha_values = np.arange(0, 0.0025, 0.00001)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(dtc, X_train, y_train, 
                                              param_name='ccp_alpha', 
                                              param_range=alpha_values, cv=cv)

# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_alpha, best_train, best_valid = get_bestParam(alpha_values, train_scores, 
                                                   valid_scores)

# Display best parameter values.
print(f'Best Alpha: {best_alpha}')
print(f'Training Accuracy: {best_train}')
print(f'Validation Accuracy: {best_valid}')

# Plot Accuracy vs Alpha for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(alpha_values, train_scores, color='blue')
plt.plot(alpha_values, valid_scores, color='red')
plt.legend(['Training Score','Validation Score'])
plt.title(f'Decision Tree Classifier - Accuracy vs Alpha')
plt.xlabel('Alpha')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
dtc = DecisionTreeClassifier(criterion='gini', random_state=random_seed, 
                             ccp_alpha=best_alpha)
dtc.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = dtc.predict(X_train)
y_test_predicted = dtc.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with ccp_alpha = {best_alpha}\n')
print('\t\tDecision Tree Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\tDecision Tree Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4))

In [ ]:
# Predict class probabilities for testing data.
y_score = dtc.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
dtc_precision, dtc_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
dtc_auc = auc(dtc_recall, dtc_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(dtc_recall, dtc_precision, color='blue')
plt.legend([f'Decision Tree (AUC = {dtc_auc})'])
plt.title(f'Decision Tree Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `ccp_alpha` value that would produce the highest testing F1 score.

The F1 score (also known as balanced F-score or F-measure) can be interpreted as a harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The F1 score is dependent upon recall and precision. The higher the recall and precision, the higher the F1 score. To increase recall and precision, I must reduce false-negatives and false-positives.

Recall = TP/(TP + FN)<br> 
Precision = TP/(TP + FP)<br> 

F1 Score = 2 x (Recall x Precision)/(Recall + Precision)

In [ ]:
fraud_train_f1_score = [] 
fraud_test_f1_score = []

for alpha in alpha_values:
    # Create model and fit model to the training data.
    dtc = DecisionTreeClassifier(criterion='gini', random_state=random_seed, 
                                 ccp_alpha=alpha)
    dtc.fit(X_train, y_train)
    
    # Predict training and testing class.
    y_train_predicted = dtc.predict(X_train)
    y_test_predicted = dtc.predict(X_test)
    
    # Calculate F1 score for training and testing data.
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_alpha, best_train, best_test = get_bestParam(alpha_values, fraud_train_f1_score, 
                                                  fraud_test_f1_score)

# Display best parameter values.
print(f'Best Alpha: {best_alpha}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Fraud F1 Score vs Alpha for testing and training data.
plt.figure(figsize=(15,8))
plt.plot(alpha_values, fraud_train_f1_score, color='blue')
plt.plot(alpha_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title(f'Decision Tree Classifier - Fraud F1 Score vs Alpha')
plt.xlabel('Alpha')
plt.ylabel(f'Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
dtc = DecisionTreeClassifier(criterion='gini', random_state=random_seed, 
                             ccp_alpha=best_alpha)
dtc.fit(X_train, y_train)

# Predict training & testing class.
y_train_predicted = dtc.predict(X_train)
y_test_predicted = dtc.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with ccp_alpha = {best_alpha}\n')
print('\t\tDecision Tree Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\tDecision Tree Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4))

In [ ]:
# Predict class probabilities for testing data.
y_score = dtc.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
dtc_precision, dtc_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
dtc_auc = auc(dtc_recall, dtc_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(dtc_recall, dtc_precision, color='blue')
plt.legend([f'Decision Tree (AUC = {dtc_auc})'])
plt.title(f'Decision Tree Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

As result of choosing an alpha that would produce the highest testing F1 score, I was able to reduce FP and FN while increasing TP & TN. 

- FP decreased by **3.21%**
- FN decreased by **9.14%**
- TP increased by **2.03%**
- TN increased by **0.18%** 

#### Training Metrics
- Precision increased by **0.66%**<br>
- Recall increased by **1.07%**<br>
- F1 score increased by **0.87%**<br>

#### Testing Metrics
- Precision increased by **0.78%**<br>
- Recall increased by **2.03**%<br>
- F1 score increased by **1.42%**<br>

I initially chose `ccp_alpha` based on the accuracy of the training and validation data. However, when choosing `ccp_alpha`  based on the F1 score, it not only improved recall and precision but it also improved the accuracy by **0.61%**. Moving forward, I will compare these two (2) methods and choose the model that performs best.

<a id="section-four"></a>
## AdaBoost Classifier
### Choosing a `n_estimators` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
# Create model
ada = AdaBoostClassifier(random_state=random_seed)

# Generates a list that starts at minimum, ends at maximum, and increments by step.
estimator_values = np.arange(1, 500, 1)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(ada, X_train, y_train, 
                                              param_name='n_estimators', 
                                              param_range=estimator_values, cv=cv)

# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_estimator, best_train, best_valid = get_bestParam(estimator_values, train_scores, 
                                                       valid_scores)

# Display best parameter values.
print(f'Best Estimator: {best_estimator}')
print(f'Training Accuracy: {best_train}')
print(f'Validation Accuracy: {best_valid}')

# Plot Accuracy vs Estimators for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(estimator_values, train_scores, color='blue')
plt.plot(estimator_values, valid_scores, color='red')
plt.legend(['Training Score','Validation Score'])
plt.title('AdaBoost Classifier - Accuracy vs Estimators')
plt.xlabel('Estimators')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
ada = AdaBoostClassifier(n_estimators=best_estimator, random_state=random_seed)
ada.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = ada.predict(X_train)
y_test_predicted = ada.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with n_estimators = {best_estimator}\n')
print('\t\t  AdaBoost Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  AdaBoost Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = ada.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
ada_precision, ada_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
ada_auc = auc(ada_recall, ada_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(ada_recall, ada_precision, color='orange')
plt.legend([f'AdaBoost (AUC = {ada_auc})'])
plt.title('AdaBoost Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `n_estimators` value that would produce the highest testing F1 score.

In [ ]:
fraud_train_f1_score = []
fraud_test_f1_score = []

for estimators in estimator_values:
    # Create model and fit model to the training data.
    ada = AdaBoostClassifier(n_estimators=estimators, random_state=random_seed)
    ada.fit(X_train, y_train)
    
    # Predict training and testing class.
    y_train_predicted = ada.predict(X_train)
    y_test_predicted = ada.predict(X_test)
    
    # Calculate F1 score for training and testing data.    
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_estimator, best_train, best_test = get_bestParam(
    estimator_values, fraud_train_f1_score, fraud_test_f1_score)

# Display best parameter values.
print(f'Best Estimator: {best_estimator}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Fraud F1 Score vs Estimators for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(estimator_values, fraud_train_f1_score, color='blue')
plt.plot(estimator_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title('AdaBoost Classifier -  Fraud F1 Score vs Estimators')
plt.xlabel('Estimators')
plt.ylabel(f'Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
ada = AdaBoostClassifier(n_estimators=best_estimator, random_state=random_seed)
ada.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = ada.predict(X_train)
y_test_predicted = ada.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with n_estimators = {best_estimator}\n')
print('\t\t  AdaBoost Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  AdaBoost Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = ada.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
ada_precision, ada_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
ada_auc = auc(ada_recall, ada_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(ada_recall, ada_precision, color='orange')
plt.legend([f'AdaBoost (AUC = {ada_auc})'])
plt.title('AdaBoost Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

<a id="section-five"></a>
## K Nearest Neighbors (KNN) Classifier
### Choosing a `n_neighbors` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# Create model
knn = KNeighborsClassifier()

# Generates a list that starts at minimum, ends at maximum, and increments by step.
neighbor_values = np.arange(1, 101, 1)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(knn, X_train, y_train, 
                                              param_name='n_neighbors', 
                                              param_range=neighbor_values, cv=cv)
# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_neighbor, best_train, best_valid = get_bestParam(neighbor_values, train_scores, 
                                                      valid_scores)

# Display best parameter values.
print(f'Best Neighbor: {best_neighbor}')
print(f'Training Accuracy: {best_train}')
print(f'Validation Accuracy: {best_valid}')

# Plot Accuracy vs Neighbors for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(neighbor_values, train_scores, color='blue')
plt.plot(neighbor_values, valid_scores, color='red')
plt.legend(['Training Score','Validation Score'])
plt.title('KNN Classifier - Accuracy vs Neighbors')
plt.xlabel('Neighbors')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
knn = KNeighborsClassifier(n_neighbors=best_neighbor)
knn.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = knn.predict(X_train)
y_test_predicted = knn.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with n_neighbors = {best_neighbor}\n')
print('\t\t     KNN Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t     KNN Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = knn.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
knn_precision, knn_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
knn_auc = auc(knn_recall, knn_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(knn_recall, knn_precision, color='magenta')
plt.legend([f'KNN (AUC = {knn_auc})'])
plt.title('KNN Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `n_neighbors` value that would produce the highest testing F1 score.

In [ ]:
fraud_train_f1_score = []
fraud_test_f1_score = []

for neighbors in neighbor_values:
    # Create model and fit model to the training data.
    knn = KNeighborsClassifier(n_neighbors=neighbors)
    knn.fit(X_train, y_train)
    
    # Predict training and testing class.
    y_train_predicted = knn.predict(X_train)
    y_test_predicted = knn.predict(X_test)
    
    # Calculate F1 score for training and testing data.    
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_neighbor, best_train, best_test = get_bestParam(
    neighbor_values, fraud_train_f1_score, fraud_test_f1_score)

# Display best parameter values.
print(f'Best Neighbors: {best_neighbor}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Fraud F1 Score vs Neighbors for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(neighbor_values, fraud_train_f1_score, color='blue')
plt.plot(neighbor_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title('KNN Classifier - Fraud F1 Score vs Neighbors')
plt.xlabel('Neighbors')
plt.ylabel('Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
knn = KNeighborsClassifier(n_neighbors=best_neighbor)
knn.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = knn.predict(X_train)
y_test_predicted = knn.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with n_neighbors = {best_neighbor}\n')
print('\t\t     KNN Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t     KNN Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = knn.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
knn_precision, knn_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
knn_auc = auc(knn_recall, knn_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(knn_recall, knn_precision, color='magenta')
plt.legend([f'KNN (AUC = {knn_auc})'])
plt.title('KNN Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

<a id="section-six"></a>
## Multi-Layer Perceptron (MLP) Classifier
### Choosing a `max_iter` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
# Create model.
mlp = MLPClassifier(hidden_layer_sizes=(10,10), activation='tanh',
                    random_state=random_seed)

# Generates a list that starts at minimum, ends at maximum, and increments by step.
max_iter_values = np.arange(10, 150, 1)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(mlp, X_train, y_train, 
                                              param_name='max_iter', 
                                              param_range=max_iter_values, cv=cv)

# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_max_iter, best_train, best_valid = get_bestParam(max_iter_values, train_scores, 
                                                      valid_scores)

# Display best parameter values.
print(f'Best Max Iterations: {best_max_iter}')
print(f'Training Accuracy: {best_train}')
print(f'Validation Accuracy: {best_valid}')

# Plot Accuracy vs Max Iterations for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(max_iter_values, train_scores, color='blue')
plt.plot(max_iter_values, valid_scores, color='red')
plt.legend(['Training Score','Validation Score'])
plt.title('MLP Classifier - Accuracy vs Max Iterations')
plt.xlabel('Max Iterations')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
mlp = MLPClassifier(hidden_layer_sizes=(10,10), activation='tanh',
                    max_iter=best_max_iter, random_state=random_seed)
mlp.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = mlp.predict(X_train)
y_test_predicted = mlp.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with max_iter = {best_max_iter}\n')
print('\t\t  MLP Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  MLP Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = mlp.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
mlp_precision, mlp_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
mlp_auc = auc(mlp_recall, mlp_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(mlp_recall, mlp_precision, color='green')
plt.legend([f'MLP (AUC = {mlp_auc})'])
plt.title('MLP Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `max_iter` value that would produce the highest testing F1 score.

In [ ]:
fraud_train_f1_score = []
fraud_test_f1_score = []

for max_iter in max_iter_values:
    # Create model and fit model to the training data.
    mlp = MLPClassifier(hidden_layer_sizes=(10,10), activation='tanh',
                    max_iter=max_iter, random_state=random_seed)
    mlp.fit(X_train, y_train)
    
    # Predict training and testing class.
    y_train_predicted = mlp.predict(X_train)
    y_test_predicted = mlp.predict(X_test)
    
    # Calculate F1 score for training and testing data.    
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_max_iter, best_train, best_test = get_bestParam(
    max_iter_values, fraud_train_f1_score, fraud_test_f1_score)

# Display best parameter values.
print(f'Best Max Iterations: {best_max_iter}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Fraud F1 Score vs Max Iterations for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(max_iter_values, fraud_train_f1_score, color='blue')
plt.plot(max_iter_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title('MLP Classifier - Fraud F1 Score vs Max Iterations')
plt.xlabel('Max Iterations')
plt.ylabel('Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
mlp = MLPClassifier(hidden_layer_sizes=(10,10), activation='tanh',
                    max_iter=best_max_iter, random_state=random_seed)
mlp.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = mlp.predict(X_train)
y_test_predicted = mlp.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with max_iter = {best_max_iter}\n')
print('\t\t  MLP Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  MLP Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = mlp.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
mlp_precision, mlp_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
mlp_auc = auc(mlp_recall, mlp_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(mlp_recall, mlp_precision, color='green')
plt.legend([f'MLP (AUC = {mlp_auc})'])
plt.title('MLP Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

<a id="section-seven"></a>
## Logistic Regression
### Choosing a `max_iter` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Create model
lrg = LogisticRegression(solver='newton-cg', random_state=random_seed)

# Generates a list that starts at minimum, ends at maximum, and increments by step.
max_iter_values = np.arange(1, 175, 1)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(lrg, X_train, y_train, 
                                              param_name='max_iter', 
                                              param_range=max_iter_values, cv=cv)

# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_max_iter, best_train, best_valid = get_bestParam(max_iter_values, train_scores, 
                                                      valid_scores)

# Display best parameter values.
print(f'Best Max Iterations: {best_max_iter}')
print(f'Training Accuracy: {best_train}')
print(f'Cross Validation Accuracy: {best_valid}')

# Plot Accuracy vs Max Iterations for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(max_iter_values, train_scores, color='blue')
plt.plot(max_iter_values, valid_scores, color='red')
plt.legend(['Training Score','Cross Validation Score'])
plt.title('Logistic Regression Classifier - Accuracy vs Max Iterations')
plt.xlabel('Max Iterations')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
lrg = LogisticRegression(solver='newton-cg', random_state=random_seed, 
                         max_iter=best_max_iter)
lrg.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = lrg.predict(X_train)
y_test_predicted = lrg.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with max_iter = {best_max_iter}\n')
print('\t\t  Logistic Regression (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  Logistic Regression (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = lrg.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
lrg_precision, lrg_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
lrg_auc = auc(lrg_recall, lrg_precision).__round__(4)

plt.figure(figsize=(9,6))            
plt.plot(lrg_recall, lrg_recall, color='blueviolet')
plt.legend([f'Logistic Regression (AUC = {lrg_auc})'])
plt.title(f'Logistic Regression Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `max_iter` value that would produce the highest testing F1 score.

In [ ]:
fraud_train_f1_score = []
fraud_test_f1_score = []

for max_iter in max_iter_values:
    # Create model and fit model to the training data.
    lrg = LogisticRegression(solver='newton-cg', random_state=random_seed, 
                             max_iter=max_iter)
    lrg.fit(X_train, y_train)
    
    # Predict training & testing class.
    y_train_predicted = lrg.predict(X_train)
    y_test_predicted = lrg.predict(X_test)
    
    # Calculate F1 score for training and testing data.    
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_max_iter, best_train, best_test = get_bestParam(
    max_iter_values, fraud_train_f1_score, fraud_test_f1_score)

# Display best parameter values.
print(f'Best Max Iterations: {best_max_iter}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Fraud F1 Score vs Max Iterations for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(max_iter_values, fraud_train_f1_score, color='blue')
plt.plot(max_iter_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title('Logistic Regression Classifier - Fraud F1 Score vs Max Iterations')
plt.xlabel('Max Iterations')
plt.ylabel(f'Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
lrg = LogisticRegression(solver='newton-cg', random_state=random_seed, 
                         max_iter=best_max_iter)
lrg.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = lrg.predict(X_train)
y_test_predicted = lrg.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with max_iter = {best_max_iter}\n')
print('\t\t  Logistic Regression (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  Logistic Regression (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = lrg.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
lrg_precision, lrg_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
lrg_auc = auc(lrg_recall, lrg_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(lrg_recall, lrg_precision, color='blueviolet')
plt.legend([f'Logistic Regression (AUC = {lrg_auc})'])
plt.title('Logistic Regression PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

<a id="section-eight"></a>
## Random Forest Classifier
### Choosing a `max_iter` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Create model
rfc = RandomForestClassifier(random_state=random_seed)

# Generates a list that starts at minimum, ends at maximum, and increments by step.
estimator_values = np.arange(1, 500, 1)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(rfc, X_train, y_train, 
                                              param_name='n_estimators', 
                                              param_range=estimator_values, cv=cv)

# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_estimator, best_train, best_valid = get_bestParam(estimator_values, train_scores, 
                                                       valid_scores)

# Display best parameter values.
print(f'Best Estimator: {best_estimator}')
print(f'Training Accuracy: {best_train}')
print(f'Validation Accuracy: {best_valid}')

# Plot Accuracy vs Estimators for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(estimator_values, train_scores, color='blue')
plt.plot(estimator_values, valid_scores, color='red')
plt.legend(['Training Score','Validation Score'])
plt.title('Random Forest Classifier - Accuracy vs Estimators')
plt.xlabel('Estimators')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
rfc = RandomForestClassifier(n_estimators=best_estimator, random_state=random_seed)
rfc.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = rfc.predict(X_train)
y_test_predicted = rfc.predict(X_test)

# Display classification report for training and testing data.
print(f'Classification Report with n_estimators = {best_estimator}\n')
print('\t\t  Random Forest Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  Random Forest Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = rfc.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
rfc_precision, rfc_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
rfc_auc = auc(rfc_recall, rfc_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(rfc_recall, rfc_precision, color='red')
plt.legend([f'Random Forest (AUC = {rfc_auc})'])
plt.title('Random Forest Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `max_iter` value that would produce the highest testing F1 score.

In [ ]:
fraud_train_f1_score = []
fraud_test_f1_score = []

for estimators in estimator_values:
    # Create model and fit model to the training data.
    rfc = RandomForestClassifier(n_estimators=estimators, random_state=random_seed)
    rfc.fit(X_train, y_train)
    
    # Predict training and testing class.
    y_train_predicted = rfc.predict(X_train)
    y_test_predicted = rfc.predict(X_test)
    
    # Calculate F1 score for training and testing data.
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_estimator, best_train, best_test = get_bestParam(
    estimator_values, fraud_train_f1_score, fraud_test_f1_score)

# Display best parameter values.
print(f'Best Estimator: {best_estimator}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Fraud F1 Score vs Estimators for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(estimator_values, fraud_train_f1_score, color='blue')
plt.plot(estimator_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title('Random Forest Classifier - Fraud F1 Score vs Estimators')
plt.xlabel('Estimators')
plt.ylabel('Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
rfc = RandomForestClassifier(n_estimators=best_estimator, random_state=random_seed)
rfc.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = rfc.predict(X_train)
y_test_predicted = rfc.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with n_estimators = {best_estimator}\n')
print('\t\t  Random Forest Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t  Random Forest Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = rfc.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
rfc_precision, rfc_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
rfc_auc = auc(rfc_recall, rfc_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(rfc_recall, rfc_precision, color='red')
plt.legend([f'Random Forest (AUC = {rfc_auc})'])
plt.title('Random Forest Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

<a id="section-nine"></a>
## Support Vector Machine (SVM)
### Choosing a `max_iter` value that would produce the highest validation accuracy score.

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Create model
svm = SVC(gamma='auto', random_state=random_seed)

# Generates a list that starts at minimum, ends at maximum, and increments by step.
max_iter_values = np.arange(1600, 3000, 20)

# Calculate training and validation accuracies.
train_scores, valid_scores = validation_curve(svm, X_train, y_train, 
                                              param_name='max_iter', 
                                              param_range=max_iter_values, cv=cv)

# Average training and validation accuracy.
train_scores = np.mean(train_scores, axis=1)
valid_scores = np.mean(valid_scores, axis=1)

# Obtain the best parameter value that would produce the highest validation accuracy.
best_max_iter, best_train, best_valid = get_bestParam(max_iter_values, train_scores, 
                                                      valid_scores)

# Display best parameter values.
print(f'Best Max Iterations: {best_max_iter}')
print(f'Training Accuracy: {best_train}')
print(f'Validation Accuracy: {best_valid}')

# Plot Accuracy vs Max Iterations for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(max_iter_values, train_scores, color='blue')
plt.plot(max_iter_values, valid_scores, color='red')
plt.legend(['Training Score','Validation Score'])
plt.title('Support Vector Classifier - Accuracy vs Max Iterations')
plt.xlabel('Max Iterations')
plt.ylabel(f'Average Accuracy of {cv}-Fold Validation')
plt.show()

In [ ]:
# Create model and fit model to the training data.
svm = SVC(gamma='auto', random_state=random_seed, 
          max_iter=best_max_iter, probability=True)
svm.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = svm.predict(X_train)
y_test_predicted = svm.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with max_iter = {best_max_iter}\n')
print('\t\t   SVM Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t   SVM Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = svm.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
svm_precision, svm_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
svm_auc = auc(svm_recall, svm_precision).__round__(4)
              
plt.figure(figsize=(9,6))
plt.plot(svm_recall, svm_precision, color='dodgerblue')
plt.legend([f'SVM (AUC = {svm_auc})'])
plt.title('SVM Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

### Choosing a `max_iter` value that would produce the highest testing F1 score.

In [ ]:
fraud_train_f1_score = []
fraud_test_f1_score = []

for max_iter in max_iter_values:
    # Create model and fit model to the training data.
    svm = SVC(gamma='auto', random_state=random_seed, max_iter=max_iter)
    svm.fit(X_train, y_train)
    
    # Predict training and testing class.
    y_train_predicted = svm.predict(X_train)
    y_test_predicted = svm.predict(X_test)
    
    # Calculate F1 score for training and testing data.
    fraud_train_f1_score.append((f1_score(y_train, y_train_predicted)).__round__(4))
    fraud_test_f1_score.append((f1_score(y_test, y_test_predicted)).__round__(4))

In [ ]:
# Obtain the best parameter value that would produce the highest testing F1 score.
best_max_iter, best_train, best_test = get_bestParam(
    max_iter_values, fraud_train_f1_score, fraud_test_f1_score)

# Display best parameter values.
print(f'Best Max Iterations: {best_max_iter}')
print(f'Training F1 Score: {best_train}')
print(f'Testing F1 Score: {best_test}')

# Plot Accuracy vs Max Iterations for training and testing data.
plt.figure(figsize=(15,8))
plt.plot(max_iter_values, fraud_train_f1_score, color='blue')
plt.plot(max_iter_values, fraud_test_f1_score, color='red')
plt.legend(['Training F1 Score','Testing F1 Score'])
plt.title('SVM Classifier - Fraud F1 Score vs Max Iterations')
plt.xlabel('Max Iterations')
plt.ylabel(f'Fraud F1 Score')
plt.show()

In [ ]:
# Create model and fit model to the training data.
svm = SVC(gamma='auto', random_state=random_seed, 
          max_iter=best_max_iter, probability=True)
svm.fit(X_train, y_train)

# Predict training and testing class.
y_train_predicted = svm.predict(X_train)
y_test_predicted = svm.predict(X_test)

# Display classification report for training & testing data.
print(f'Classification Report with max_iter = {best_max_iter}\n')
print('\t\t   SVM Classifier (Train)')
print('\t---------------------------------------------')
print(classification_report(y_train, y_train_predicted, 
                            target_names=target_names, digits=4), '\n')
print('\t\t   SVM Classifier (Test)')
print('\t---------------------------------------------')
print(classification_report(y_test, y_test_predicted, 
                            target_names=target_names, digits=4), '\n')

In [ ]:
# Predict class probabilities for testing data.
y_score = svm.predict_proba(X_test)[:, 1]

# Compute precision-recall pairs for different probability thresholds.
svm_precision, svm_recall, _ = precision_recall_curve(y_test, y_score)

# Calculate and plot AUC-PR.
svm_auc = auc(svm_recall, svm_precision).__round__(4)
              
plt.figure(figsize=(9,6))            
plt.plot(svm_recall, svm_precision, color='dodgerblue')
plt.legend([f'SVM (AUC = {svm_auc})'])
plt.title('SVM Classifier PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

In [ ]:
save_metrics()

<a id="section-ten"></a>
# Comparing ML Models

I will be comparing different metrics (e.g., accuracy, F1 score and AUC-PR) to determine which model is the most efficient at predicting fraudulent transactions. 

In [ ]:
from operator import itemgetter # Used in sorting procedure.
import seaborn as sns
sns.set(style="whitegrid")

### Model vs Accuracy

Compare training and testing accuracies for all the models.

In [ ]:
D = [] # Used to create list of tuples.

# Create list of tuples with classifer name, training accuracy, and testing accuracy.
for i in range(len(clf_names)):
    D.append((clf_names[i], train_accs[i], test_accs[i]))

# Sort list of tuples by test accuracy.
Dsort = sorted(D, key=itemgetter(2), reverse=False) 

clf_names_0 = [x[0] for x in Dsort] # Recreate list with sorted classifier (model) names.
train_accs = [x[1] for x in Dsort] # Recreate list with sorted training accuracies. 
test_accs = [x[2] for x in Dsort] # Recreate list with sorted testing accuracies. 

ind = np.arange(len(clf_names_0)) # Barchart index
width = 0.28 # Barwidth

plt.figure(figsize=(15,8))
ax = plt.subplot(111)
ax.barh(ind, train_accs, width, align='center', color='pink', label='Training') 
ax.barh(ind - width, test_accs, width, align='center', color='red', label='Testing') 
ax.set(yticks=ind - width/2, yticklabels=clf_names_0, ylim=[2*width - 1, len(clf_names_0)])
plt.xlabel('Accuracy')
plt.title('Fraud Detection - Model vs Accuracy')
plt.legend()
plt.savefig('img/fraud_detection_model_accuracy.png', bbox_inches='tight')
plt.show()

The **Random Forest** model has the highest training and testing accuracy. The training accuracy is **0.9999** and the testing accuracy is **0.9323**.

### Model vs F1 Fraud Score

Compare training and testing fraud F1 scores for all the models.

In [ ]:
D = [] # Used to create list of tuples.

for i in range(len(clf_names)):
    D.append((clf_names[i], train_f1_scores[i], test_f1_scores[i]))
       
Dsort = sorted(D, key=itemgetter(2), reverse=False) # Sort the list by test F1 score.

clf_names_1 = [x[0] for x in Dsort] # Recreate list with sorted classifier (model) names.
train_f1_scores = [x[1] for x in Dsort] # Recreate list with sorted false positives. 
test_f1_scores = [x[2] for x in Dsort] # Recreate list with sorted false negatives. 
  
ind = np.arange(len(clf_names_1)) # Barchart index
width = 0.28 

plt.figure(figsize=(15,8))
ax = plt.subplot(111)
ax.barh(ind, train_f1_scores, width, align='center', color='#D4B9DA', label='Training') 
ax.barh(ind - width, test_f1_scores, width, align='center', color='#7A0177', label='Testing') 
ax.set(yticks=ind - width/2, yticklabels=clf_names_1, ylim=[2*width - 1, len(clf_names_1)])
plt.xlabel('Fraud F1 Score')
plt.title('Fraud Detection - Model vs Fraud F1 Score')
plt.legend()
plt.savefig('img/fraud_f1_score.png', bbox_inches='tight')
plt.show()

The **Random Forest** model has the highest training and testing fraud F1 score. The training fraud F1 score is **0.9998** and the testing fraud F1 score is **0.8653**.

### False Values vs Model

Compare the testing data false-positives and false-negatives for each model.

In [ ]:
D = [] # Used to create list of tuples.

for i in range(len(clf_names)):
    D.append((clf_names[i], false_pos[i], false_neg[i], false_pos[i]+false_neg[i]))

# Sort the list by total false-postives and false-negatives.
Dsort = sorted(D, key=itemgetter(3), reverse=False) 

clf_names_2 = [x[0] for x in Dsort] # Recreate list with sorted classifier (model) names.
false_pos = [x[1] for x in Dsort] # Recreate list with sorted false positives. 
false_neg = [x[2] for x in Dsort] # Recreate list with sorted false negatives. 
  
# Plot bars in stack manner
plt.figure(figsize=(15,8))
plt.bar(clf_names_2, false_pos, color='green')
plt.bar(clf_names_2, false_neg, bottom=false_pos, color='lightgreen')
plt.xlabel('Model')
plt.ylabel('False Values')
plt.title('Fraud Detection - False Values vs Model')
plt.legend(['False Postives', 'False Negatives'])
plt.savefig('img/fraud_detection_false_values.png', dpi=300, bbox_inches='tight')
plt.show()

The **Random Forest** model has the least amount of misclassified transactions with **84 false-positives** and **193 false-negatives**. This means that this model misclassifed **277 transactions out of the 4094 transactions** (6.766% error) in the testing data.

### AUC-PR: Area Under The (Precision-Recall) Curve

Compare the testing data AUC-PR for each model.

In [ ]:
plt.figure(figsize=(15,8))
plt.plot(rfc_recall, rfc_precision, color='red')
plt.plot(ada_recall, ada_precision, color='orange')
plt.plot(lrg_recall, lrg_precision, color='blueviolet')
plt.plot(dtc_recall, dtc_precision, color='blue')
plt.plot(mlp_recall, mlp_precision, color='green')
plt.plot(svm_recall, svm_precision, color='dodgerblue')
plt.plot(knn_recall, knn_precision, color='magenta')
plt.plot(dum_recall, dum_precision, color='black', linestyle='--')
plt.legend([f'Random Forest (AUC = {rfc_auc})', 
            f'AdaBoost (AUC = {ada_auc})',
            f'Logistic Regression (AUC = {lrg_auc})', 
            f'Decision Tree (AUC = {dtc_auc})',
            f'MLP (AUC = {mlp_auc})', 
            f'SVM (AUC = {svm_auc})',
            f'KNN (AUC = {knn_auc})', 
            f'Dummy (AUC = {dum_auc})'])
plt.title('PR Curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.savefig('img/PR_Curve.png', dpi=300, bbox_inches='tight')
plt.show()

Once again, **Random Forest** performed the best with the highest AUC-PR.

<a id="section-eleven"></a>
# Conclusion

All the models performed better than the baseline, but the **Random Forest** model performed the best overall. It has the highest accuracy, F1 score, AUC-PR and it has the least amount of cumulative false values (FP + FN). The **AdaBoost** and **Decision Tree** models are also great choices as well. They both have less false-negatives than the **Random Forest** model, which in my opinion is more consequential than false-positives.